<h4> Save for Storyboard </h4>

In [337]:
import struct, socket
import numpy as np 
import csv, json 
import os 
import urllib2 
import datetime
import operator

#import ipywidgets as widgets # if your using jupyter/ipython < 1.4 please comment this line and uncomment the lines below
from IPython.html import widgets
from IPython.html.widgets import interact, interactive, fixed
from IPython.display import display, HTML, clear_output, Javascript 


path = os.getcwd().split("/")
date = path[len(path)-1] 
spath = os.getcwd() 
data_src = 'http://localhost:8895/app/api/dns/threat_investigation/' 
content_type = {'Content-Type':'application/json'}
anchor = ''
anchor_type = ''

**Interface**

In [338]:
##Expanded search
susp_box   =  widgets.HBox(width=500, height=150)    
susp_h_box =  widgets.Box(width=500, height=200)    
susp_title =  widgets.HTML(value='<h4>Expanded search</h4>')
susp_select = widgets.Select(width=300)
search_btn = widgets.Button(description='Search')
susp_box.children = [susp_select,search_btn]
susp_h_box.children = [susp_title,susp_box] 

##Search Results
result_title = widgets.HTML(value='<h4>Search results</h4>')     
result_html = widgets.HTML(width=500)
result_box  = widgets.Box(width=500)
result_box.children = [result_title, result_html]               

#Threat Summary
tc_box_main = widgets.Box(width=500, height=200)    
threat_container = widgets.HBox(width=500, height=150)
tc_box_separator = widgets.Box(width=500, height=15)     
tc_div_label = widgets.HTML()            
tc_txt_title = widgets.Text(value='',width=300, placeholder='Threat Title')
tc_txa_summary = widgets.Textarea(value='', width=300, height=100)
tc_btn_save = widgets.Button(description='Save')
threat_container.children = [tc_txa_summary,tc_btn_save]
tc_box_main.children = [tc_div_label, tc_txt_title, tc_box_separator, threat_container]  

def start_investigation():
    ips_query = {}  
    url = data_src + date
    reader = json.loads(urllib2.urlopen(url).read())
    display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
       
    for row in reader['data']:    
        if row['ip_sev'] == 1: 
            ips_query[row['ip_dst']]='i'
        if row['dns_sev'] == 1:
            ips_query[row['ip_dst']]='q' 
            
    if len(ips_query) == 0:
        result_html = widgets.HTML(value="There are not high risk results.", width=500)
        result_box  = widgets.Box(width=500, height=150)
        result_box.children = [result_html]        
        display(result_box)              
    else:  
        sorted_dict = sorted(ips_query.items(), key=operator.itemgetter(0))      
        display_controls(sorted_dict)   

        
def display_controls(ip_list): 
    susp_select.options = ip_list
    susp_select.height=min(len(ip_list)*18,150)
    susp_select.selected_label = ip_list[0][0]
    display(susp_h_box)
  
    def search_ip(b):  
        global anchor 
        global anchor_type
        anchor = ''
        anchor_type = ''
        anchor = susp_select.selected_label  
        anchor_type = susp_select.value 
        removeWidget(2)
        removeWidget(1) 
            
        table = "<table><th>IP</th><th>QUERY</th><th>TOTAL</th>"
         
        if anchor_type == 'i':
            params  = '{"ip_dst":"'+ anchor + '"}'
        elif anchor_type == 'q':
            params  = '{"dns_qry_name":"'+ anchor + '"}'
            
        url = data_src + "visual/" + date 
        request = urllib2.Request(url, params, content_type)
        results = json.loads(urllib2.urlopen(request).read())         
        
        if len(results['data']) > 0:
            for row in results['data']:  
                table += "<tr><td>" +  row['ip_dst'] + "</td><td>" + row['dns_qry_name'] + "</td><td>" + str(row['total']) + "</td></tr>"  
            table += "</table>"       
        else: 
            table = "No additional threats found for {0}".format(anchor)
        
        result_html.value=table
        result_box.height=max(200,len(results['data'])*25)
        display_threat_box(anchor)
        display(result_box)
    
    search_btn.on_click(search_ip)

        
def display_threat_box(ip):    
    tc_div_label.value='<h4>Threat summary for ' + anchor +'</h4>'
    display(tc_box_main)     
    
    def save_threat_summary(b):
        global anchor
        anchor_ip =''
        anchor_dns ='' 
        if anchor != '':
            file_name = spath + "/threat_comments.csv" 
            if anchor_type == 'i':
                anchor_ip = anchor
            elif anchor_type == 'q':
                anchor_dns = anchor
                
            dt = str(datetime.datetime.now())
            yy = date[0:4]
            mm = date[4:6]
            dd = date[6:8]
            usr = "IPython"
    
            with open(file_name, 'a') as comment:
                comment.write(anchor_ip + '|' + anchor_dns + '|' + tc_txt_title.value + '|' + tc_txa_summary.value.replace('\n', '\\n') + '|' + dt + '|' + usr + '|' + yy + '|' + mm + '|' + dd + "\n")                 
            url = data_src + "comments/" + date  
            params  = '{"data":"' + file_name + '"}'  
            results = urllib2.urlopen(urllib2.Request(url, params, content_type)).read()
            
            if results:
                os.remove(file_name)               
                removeWidget(2)
                removeWidget(1) 
                display(Javascript("$(\"option[data-value='" + anchor  +"']\").remove();"))    
                response = "Successfully saved"
            else:
                response = results 
        else:
            response = "No data selected"
            
        save_html = widgets.HTML(value=response, width=500)
        save_box  = widgets.Box(width=500, height=150)
        save_box.children = [save_html]  
        susp_select.selected_label = susp_select.options[0][0]
        display(save_box)     
        
        
    tc_btn_save.on_click(save_threat_summary)
    

def removeWidget(index):
    js_command = "var widget_list = $('.widget-area').children('.widget-subarea').children('.widget-box'); \n"
    js_command = js_command + "if(widget_list.length > {0} ) \n ".format(index)
    js_command = js_command + "widget_list[{0}].remove(); \n ".format(index)
    
    display(Javascript(js_command))  

In [339]:
start_investigation()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>